<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/word2vec_clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a clasificar el sentimiento de reviews de películas con **Word2Vec**, en particular, con una red feed-forward que toma los embeddings de las palabras Word2Vec como entradas.

TODO: pasar las partes sobre similitud a otra notebook.

### Configuración del entorno

In [2]:
!pip install -qU datasets gensim spacy watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.4 which is incompatible.


In [3]:
%reload_ext watermark

In [4]:
%watermark -vmp datasets,gensim,spacy,torch,numpy,pandas,tqdm

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 7.34.0

datasets: 3.3.1
gensim  : 4.3.3
spacy   : 3.8.4
torch   : 2.5.1+cu124
numpy   : 1.26.4
pandas  : 2.2.2
tqdm    : 4.67.1

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.85+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



Para usar GPU, arriba a la derecha seleccionar "Change runtime type" --> "T4 GPU".

Es un buena idea desarrollar con CPU, y usar GPU para la corrida final, para que Google no nos limite el uso. En esta notebook vamos a trabajar con pocos datos y un modelo chico, por lo cual no es imprescindible usar GPU.

In [29]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Dataset

Cargamos y exploramos el dataset de [reviews de películas de Rotten Tomatoes](https://huggingface.co/datasets/rotten_tomatoes).

In [5]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [7]:
label_names = dataset["train"].features["label"].names
print(label_names)

['neg', 'pos']


In [8]:
import pandas as pd
import numpy as np
import datasets
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    """Muestra num_examples ejemplos aleatorios del dataset.
    """
    indices = np.random.randint(0, len(dataset), num_examples)
    df = pd.DataFrame(dataset[indices])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

np.random.seed(33)
show_random_elements(dataset["train"], num_examples=6)

,text,label
0,"drives for the same kind of bittersweet , conciliatory tone that three seasons achieved but loses its way in rhetorical excess and blatant sentimentality .",neg
1,"until it goes off the rails in its final 10 or 15 minutes , wendigo , larry fessenden's spooky new thriller , is a refreshingly smart and newfangled variation on several themes derived from far less sophisticated and knowing horror films .",pos
2,"excessive , profane , packed with cartoonish violence and comic-strip characters .",neg
3,haynes has so fanatically fetishized every bizarre old-movie idiosyncrasy with such monastic devotion you're not sure if you should applaud or look into having him committed .,pos
4,"singer/composer bryan adams contributes a slew of songs  a few potential hits , a few more simply intrusive to the story  but the whole package certainly captures the intended , er , spirit of the piece .",pos
5,a refreshing korean film about five female high school friends who face an uphill battle when they try to take their relationships into deeper waters .,pos


In [9]:
print("Distribucion de clases:")
for k in dataset.keys():
    print(k)
    labels = dataset[k]["label"]
    print(pd.Series(labels).value_counts())
    print("-"*70)

Distribucion de clases:
train
1    4265
0    4265
Name: count, dtype: int64
----------------------------------------------------------------------
validation
1    533
0    533
Name: count, dtype: int64
----------------------------------------------------------------------
test
1    533
0    533
Name: count, dtype: int64
----------------------------------------------------------------------


In [10]:
print("Largo de los documentos (en palabras), deciles:")
for k in dataset.keys():
    print(k)
    textos = dataset[k]["text"]
    largos = pd.Series(textos).str.split().apply(len)
    # print(largos.describe())
    print(np.quantile(largos, q=np.arange(0, 1.1, .1)).astype(int))
    print("-"*70)

Largo de los documentos (en palabras), deciles:
train
[ 1  9 12 15 18 20 23 26 29 34 59]
----------------------------------------------------------------------
validation
[ 1  8 12 16 18 21 23 26 29 34 54]
----------------------------------------------------------------------
test
[ 3  9 13 15 18 20 23 26 29 34 52]
----------------------------------------------------------------------


**PREGUNTA**: ¿para qué sirve mirar la distribución de clases?

## Embeddings word2vec

Vamos a extraer embeddings de las palabras con Word2Vec y entrenar una red feed-forward con una capa oculta.

Usamos `gensim` para extraer los embeddings y `pytorch` para entrenar la red. Luego, cuando usemos transformers vamos a usar funciones de la librería `transformers` de HF que ayudan a automatizar algunas tareas.

In [11]:
import gensim.downloader as api

api.info().keys()

dict_keys(['corpora', 'models'])

In [12]:
# modelos disponibles:
api.info()["models"].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

Cargamos los embeddings pre-entrenados, y analizamos el vocabulario y los embeddings de las palabras.

**PREGUNTA**: ¿qué diferencias hay _grosso modo_ entre estos modelos? ¿qué quiere decir que son pre-entrenados?

In [13]:
# puede demorar unos 10 minutos...
wv_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [14]:
print("Tamaño del vocabulario:")
print(len(wv_model.index_to_key))

print("Palabras más frecuentes:")
print(wv_model.index_to_key[:20])

print("Palabras menos frecuentes:")
print(wv_model.index_to_key[-20:])

Tamaño del vocabulario:
3000000
Palabras más frecuentes:
['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said', 'was', 'the', 'at', 'not', 'as', 'it', 'be', 'from', 'by', 'are']
Palabras menos frecuentes:
['malfunctioning_tin_whisker', 'Frequency_Converters', 'Commando_Battalions', 'Audi_A4_saloon', 'Cow_Plop_Bingo', 'sheepdog_herding', 'Shahid_Afridi_Rana_Naved', 'Anagnostaras_Adams', 'chairman_Ken_Dulieu', 'USA_TODAY_Reid_Cherner', 'Mauri_Liberati', 'Shilpa_Goenka', 'divider_pistons', 'Thirsty_Owl', 'righthander_Kyle_Drabek', 'RAFFAELE', 'Bim_Skala_Bim', 'Mezze_Cafe', 'pulverizes_boulders', 'snowcapped_Caucasus']


In [15]:
tokens = ["the", "The", ".", ",", "and", "xeneize", "<unk>", "<pad>"]

for token in tokens:
    print(f"{token:10s} in wv vocab: {token in wv_model}")

the        in wv vocab: True
The        in wv vocab: True
.          in wv vocab: False
,          in wv vocab: False
and        in wv vocab: False
xeneize    in wv vocab: False
<unk>      in wv vocab: False
<pad>      in wv vocab: False


In [16]:
# Extraemos el vector de una palabra:
vec = wv_model["hello"]
print(vec.shape)
print(wv_model.vector_size)
print(vec[:5])

(300,)
300
[-0.05419922  0.01708984 -0.00527954  0.33203125 -0.25      ]


Podemos usar la similitud coseno para medir la cercanía entre palabras.

In [17]:
import numpy as np

def cossim(v1, v2):
    """Calcula la similitud coseno entre dos vectores v1 y v2.
    Args:
        v1: vector 1
        v2: vector 2
    """
    producto_interno = np.dot(v1, v2)
    norma_v1 = np.linalg.norm(v1)
    norma_v2 = np.linalg.norm(v2)
    return producto_interno / (norma_v1 * norma_v2)


print("Prueba:")
vec_banana = wv_model["banana"]
vec_apple = wv_model["apple"]
print(f"{cossim(vec_banana, vec_apple) = :.4f}")

Prueba:
cossim(vec_banana, vec_apple) = 0.5318


**PREGUNTA**: es una consigna en realidad: elegir una palabra en inglés ($w$), y otras tres que estén semánticamente asociadas en orden decreciente según la fuerza de la asociación ($x_1$,$x_2$,$x_3$). Calcular la similitud coseno entre $w$ y $x_i$ y verificar que la similitud en el espacio de los embeddings correlaciona con la similitud semántica _a priori_. ¿Cómo hizo word2vec para capturar estas relaciones?

In [18]:
# w = ...
# x1 = ...
# x2 = ...
# x3 = ...

# print(f"{cossim(..., ...) = :.4f}")
# print(f"{cossim(..., ...) = :.4f}")
# print(f"{cossim(..., ...) = :.4f}")

In [19]:
def most_similar_by_vector(vec, top=None) -> dict:
    """Hallar las palabras más similares al vector vec.
    Args:
        vec: vector
        top: número de palabras más similares a devolver
    Imita el comportamiento de model.similar_by_vector(vec, topn=top)
    Implementación ingenua que no paraleliza con multiplicación de matrices.
    """
    similarities = {}
    for w in wv_model.index_to_key:
        similarities[w] = cossim(wv_model[w], vec)
    sorted_similarities = dict(sorted(similarities.items(), key=lambda item: item[1], reverse=True))
    if top is not None:
        return dict(list(sorted_similarities.items())[:top])
    else:
        return sorted_similarities

def most_similar(word, top=None) -> list:
    """Hallar las palabras más similares a una palabra dada.
    Args:
        word: palabra
        top: número de palabras más similares a devolver
    Imita a model.most_similar(word, topn=top)
    """
    similarities = most_similar_by_vector(wv_model[word], top=None)
    similarities.pop(word, None)
    if top is not None:
        return list(similarities.items())[:top]
    else:
        return list(similarities.items())

In [20]:
np.random.seed(33)
random_vec = np.random.normal(size=wv_model.vector_size)
print(most_similar_by_vector(random_vec, top=5))

{'Dismount': 0.3143983186929963, 'Naming_Contest': 0.3033102125629838, 'MicroArray': 0.29091885853060184, 'Bivy': 0.289917005820241, 'H6': 0.2886864831536217}


In [21]:
print(wv_model.most_similar("banana", topn=5))

[('bananas', 0.7523775696754456), ('pineapple', 0.6587537527084351), ('mango', 0.6365211009979248), ('pineapples', 0.6313878297805786), ('papaya', 0.6063665747642517)]


In [22]:
print(most_similar("banana" , top=5))

[('bananas', 0.75237757), ('pineapple', 0.6587538), ('mango', 0.63652116), ('pineapples', 0.6313878), ('papaya', 0.60636663)]


**PREGUNTA**: ¿0.3 es una similitud alta o baja? ¿Y 0.7? ¿Y 0.9? ¿De qué depende esto?

>La idea es la siguiente:
>
>1. Hacer un **vocabulario** con las palabras del dataset de train. Usamos un corte por frecuencia para determinar cuándo una palabra es desconocida (token especial "\<unk\>"). También agregamos un token especial "\<pad\>" para rellenar las secuencias en el armado de los batches.
>2. **Tokenizamos** las reviews y las convertimos en secuencias de índices. Cada índice representa una palabra dentro del vocabulario.
>3. Armamos una **matriz de embeddings** de dimensión (n_vocab, embedding_dim) donde cada fila es el embedding de una palabra del vocabulario. Si la palabra está en el vocab de word2vec, usamos ese embedding; caso contrario, inicializamos aleatoriamente.
>4. Armamos **batches** de secuencias de índices con **padding** i.e. rellenando las secuencias más cortas con el token "\<pad\>" para que todas tengan la misma longitud dentro del batch.
>5. Definimos una **red de clasificación binaria** que toma como entrada un batch de secuencias de índices de dimensión (batch_size, max_seq_len) y devuelve un tensor de probabilidades de dimensión (batch_size, 2).
>6. **Entrenamos** el modelo de clasificación lineal reajustando los embeddings y evaluamos su rendimiento.

## Construcción del vocabulario y tokenización

Para pasar de documentos a tokens y construir el vocabulario, usamos simplemente `split()` de Python. Vamos a considerar como parte del vocabulario todas las palabras que ocurran al menos dos veces.

In [23]:
def create_vocab(dataset, min_frec=2):
    """Crea un vocabulario a partir de un dataset de Hugging Face.
    Args:
        dataset: una partición de un dataset de Hugging Face
    Returns:
        Dos diccionarios: token2idx (palabra -> índice) y idx2token (índice -> palabra)
    """
    str2count = {}
    for example in dataset["train"]:
        for token in example["text"].split():
            str2count[token] = str2count.get(token, 0) + 1
    # filtrar por min_frec:
    str2count = {token: count for token, count in str2count.items() if count >= min_frec}
    # ordenar de mayor a menor frecuencia:
    str2count = dict(sorted(str2count.items(), key=lambda x: x[1], reverse=True))
    # Mapeamos cada token a un índice distinto
    token2idx = {token: idx for idx, token in enumerate(str2count)}
    # Agregamos "<unk>" y "<pad>" al vocab:
    token2idx["<unk>"] = len(str2count)
    token2idx["<pad>"] = len(str2count) + 1
    # "Invertir" el diccionario:
    idx2token = {idx: token for idx, token in enumerate(token2idx)}
    return token2idx, idx2token


token2idx, idx2token = create_vocab(dataset)

In [24]:
print(f"vocab_size = {len(token2idx)}")
print(f"1ros tokens = {list(token2idx.keys())[:10]}")
print(f"úlimos tokens = {list(token2idx.keys())[-10:]}")

vocab_size = 8981
1ros tokens = ['.', 'the', ',', 'a', 'and', 'of', 'to', 'is', 'in', 'that']
úlimos tokens = ['melodramas', 'reactionary', 'heyday', 'denouement', 'snore', 'bray', 'glee', 'secondhand', '<unk>', '<pad>']


Tokenizamos los documentos de todo el dataset (convertimos los strings en listas de índices).

In [25]:
# Tokenizamos todos los splits del dataset:
def tokenize(example):
    """Procesa un example de un dataset de Hugging Face, tokenizando el texto
    y convirtiendo tokens a ids. Agrega una columna "input_ids" al example con
    un array de enteros.
    """
    token_ids = [token2idx.get(token, token2idx["<unk>"]) for token in example["text"].split()]
    return {"input_ids": np.array(token_ids, dtype=np.int64)}

dataset = dataset.map(tokenize)
dataset

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids'],
        num_rows: 1066
    })
})

In [26]:
# en cada split:
# Calculamos la proporción de "<unk>" y de tokens que no están en w2v vocab.
for k in dataset.keys():
    print(k)
    all_input_ids = np.hstack(dataset[k]["input_ids"])
    all_tokens = [idx2token[idx] for idx in all_input_ids]
    token_counts = pd.Series(all_input_ids).value_counts(normalize=True)
    unk_count = token_counts.get(token2idx["<unk>"], 0)
    print(f"{unk_count = :.2%}")
    not_in_wv_count = np.mean([token not in wv_model for token in all_tokens])
    print(f"{not_in_wv_count = :.2%}")
    print("-"*70)

train
unk_count = 5.57%
not_in_wv_count = 30.13%
----------------------------------------------------------------------
validation
unk_count = 9.28%
not_in_wv_count = 33.27%
----------------------------------------------------------------------
test
unk_count = 8.88%
not_in_wv_count = 32.99%
----------------------------------------------------------------------


In [27]:
# Veamos un ejemplo:
example = dataset["validation"][0]
for token, id_ in zip(example["text"].split(), example["input_ids"]):
    in_wv = token in wv_model
    print(f"{token:16s} --> {id_:6d} (in wv: {in_wv})")

compassionately  -->   8979 (in wv: True)
explores         -->   2818 (in wv: True)
the              -->      1 (in wv: True)
seemingly        -->   2427 (in wv: True)
irreconcilable   -->   8979 (in wv: True)
situation        -->    835 (in wv: True)
between          -->    153 (in wv: True)
conservative     -->   3579 (in wv: True)
christian        -->   2129 (in wv: True)
parents          -->    673 (in wv: True)
and              -->      4 (in wv: False)
their            -->     64 (in wv: True)
estranged        -->   8979 (in wv: True)
gay              -->   1137 (in wv: True)
and              -->      4 (in wv: False)
lesbian          -->   8979 (in wv: True)
children         -->    463 (in wv: True)
.                -->      0 (in wv: False)


## Matriz de embeddings

Preparamos la matriz de embeddings. Si un token está en el vocab de w2v, inicializamos con ese vector. Caso contrario, usamos inicialización random. Para "\<pad\>" usamos 0s, y para "\<unk\>", el promedio de todos los embeddings.

In [30]:
def create_embedding_matrix(wv_model, token2idx):
    """Crea una matriz de embeddings de dimensión (vocab_size, embedding_dim)
    donde cada fila es el embedding de una palabra del vocabulario.
    Args:
        wv_model: modelo word2vec
        token2idx: vocab (diccionario que mapea tokens a índices)
    """
    vocab_size = len(token2idx)
    embedding_dim = wv_model.vector_size
    embedding_matrix = torch.randn(vocab_size, embedding_dim) # random normal(0, 1)
    for token, idx in token2idx.items():
        if token in wv_model:
            embedding_matrix[idx] = torch.tensor(wv_model[token])
    embedding_matrix[token2idx["<pad>"]] = torch.zeros(embedding_dim)
    embedding_matrix[token2idx["<unk>"]] = embedding_matrix.mean(dim=0)
    return embedding_matrix


torch.manual_seed(33)
embedding_matrix = create_embedding_matrix(wv_model, token2idx)
print("Primeros valores de la matriz de embeddings:")
print(embedding_matrix[:5, :5])

Primeros valores de la matriz de embeddings:
tensor([[ 0.6014,  0.1087, -0.4499,  0.7841,  0.6550],
        [ 0.0801,  0.1050,  0.0498,  0.0535, -0.0674],
        [ 0.3890, -0.3511, -0.5480,  0.2813, -0.2337],
        [ 0.6605,  1.0803,  1.0876, -0.1662, -0.2425],
        [-0.0683,  0.6505,  0.3341, -0.8173,  1.4925]])


**PREGUNTA**: ¿qué dimensiones tiene la matriz de embeddings? ¿Qué representan las filas y las columnas?

## Armado de _batches_

Armamos los batches para entrenar y para evaluar el modelo. Para esto

* Usamos la clase `DataLoader` de PyTorch. En cada iteración, el `DataLoader` nos devuelve un batch de ejemplos.
* Definimos una función `collate_fn` que se encarga de preparar los batches. En este caso, se encarga de rellenar las secuencias con el token "\<pad\>" para que todas tengan la misma longitud dentro de un batch.

**PREGUNTA**: ¿por qué hacemos procesamiento en batches?

In [31]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def collate_fn(examples):
    """Función que se pasa a DataLoader para que se encargue de hacer padding
    de las secuencias.
    Devuelve un diccionario con los tensores de las secuencias de texto,
    las longitudes de las secuencias y las etiquetas. Necesitamos las longitudes
    para poder hacer pooling en la capa de embeddings de manera correcta.
    """
    input_ids = [example["input_ids"] for example in examples]
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=token2idx["<pad>"])
    # batch_first=True: devuelve tensor con batch_size como 1ra dim.
    inputs_lengths = torch.tensor([len(example["input_ids"]) for example in examples], dtype=torch.long)
    labels = torch.tensor([example["label"] for example in examples], dtype=torch.long)
    return {"input_ids": input_ids, "inputs_lengths": inputs_lengths, "labels": labels}

In [32]:
batch_size = 32
dataset.set_format("torch", columns=["input_ids", "label"])

train_loader = DataLoader(dataset["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(dataset["validation"], batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(dataset["test"], batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [33]:
# Veamos los primeros dos batches de entrenamiento:
torch.manual_seed(33)
for i, data in enumerate(train_loader):
    print(f"batch {i}")
    print(f"Shapes = {[s.shape for s in data.values()]}")
    print(data)
    if i == 1:
        break

batch 0
Shapes = [torch.Size([32, 43]), torch.Size([32]), torch.Size([32])]
{'input_ids': tensor([[4318,  294,   17,  ...,   26,  742,    0],
        [   3, 2078,    2,  ..., 8980, 8980, 8980],
        [  14, 1228,  181,  ..., 8980, 8980, 8980],
        ...,
        [ 268,  583,   31,  ..., 8980, 8980, 8980],
        [  13, 3922,  891,  ..., 8980, 8980, 8980],
        [   0,    0,    0,  ..., 8980, 8980, 8980]]), 'inputs_lengths': tensor([43, 16, 26, 33, 23, 11,  9, 17, 13, 15, 20, 18, 12, 28, 12, 36, 19,  7,
        29, 19, 18, 18, 17, 14,  7, 21, 34,  8, 23, 26, 17,  5]), 'labels': tensor([0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
        0, 1, 1, 0, 1, 1, 1, 0])}
batch 1
Shapes = [torch.Size([32, 37]), torch.Size([32]), torch.Size([32])]
{'input_ids': tensor([[  44,   45, 1226,  ..., 8980, 8980, 8980],
        [  10,  170,   28,  ..., 8980, 8980, 8980],
        [ 157,    5,   17,  ..., 8980, 8980, 8980],
        ...,
        [ 108,   29,  104,  ..., 898

Definimos la red neuronal.

In [34]:
from torch import nn

class FFNModel(nn.Module):
    """Red Feed-Forward de dos capas (hidden layer + output layer).
    """
    def __init__(self, vocab_size, embedding_dim, pad_idx, hidden_size, num_labels,
                 embedding_matrix=None, freeze_embeddings=False):
        """Args:
            vocab_size: tamaño del vocabulario
            embedding_dim: dimensión de los embeddings
            pad_idx: índice del token "<pad>"
            hidden_size: dimensión de la capa oculta
            num_labels: número de clases de salida
            embedding_matrix: matriz de embeddings pre-entrenadas
            freeze_embeddings: si True, no se entrena la matriz de embeddings
        """
        super().__init__()
        if embedding_matrix is not None:
            self.embedding = nn.Embedding.from_pretrained(
                embedding_matrix, padding_idx=pad_idx,
                freeze=freeze_embeddings
            )
        else:
            self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(embedding_dim, hidden_size)
        self.clf = nn.Linear(hidden_size, num_labels)

    def forward(self, text, text_lengths):
        """Dimensiones de los tensores:
            text: (batch_size, seq_len)
            text_lengths: (batch_size,)
        Returns:
            Un tensor de **logits** (batch_size, num_labels)
        """
        embeddings = self.embedding(text) # (batch_size, seq_len, embedding_dim)
        pooled_embeddings = embeddings.sum(dim=1) / text_lengths.unsqueeze(1) # (batch_size, embedding_dim)
        z = torch.relu(self.fc(pooled_embeddings)) # (batch_size, hidden_size)
        logits = self.clf(z) # (batch_size, num_labels)
        return logits

**PREGUNTA**: ¿cómo hacemos para procesar textos de distinta longitud?

**PREGUNTA**: ¿qué hace esto `embeddings.sum(dim=1) / text_lengths.unsqueeze(1)`?

**PREGUNTA**: ¿qué va a hacer el modelo si aparece un token nuevo/desconocido (OOV) en la inferencia? ¿Y si es un token conocido pero que no tiene embedding w2v?

## Entrenamiento

In [35]:
ffn_model = FFNModel(
    vocab_size=len(token2idx),
    embedding_dim=wv_model.vector_size,
    pad_idx=token2idx["<pad>"],
    hidden_size=128,
    num_labels=dataset["train"].features["label"].num_classes,
    embedding_matrix=embedding_matrix,
    freeze_embeddings=False
)

Definimos `CrossEntropyLoss` como función de pérdida (está diseñada para trabajar con los logits de salida del modelo, _no_ las probabilidades), y el optimizador Adam, y una función para computar las métricas de evaluación que nos interesan.

In [36]:
from torch import optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(ffn_model.parameters(), lr=1e-3)

In [37]:
def compute_metrics(logits, labels):
    """Args:
        logits: array shape (batch_size, num_labels)
        labels: array shape (batch_size,)
    """
    # Usamos torch para usar loss_fn, pero podriamos usar cpu y numpy
    if not isinstance(logits, torch.Tensor):
        logits = torch.tensor(logits)
    if not isinstance(labels, torch.Tensor):
        labels = torch.tensor(labels)
    predictions = torch.argmax(logits, dim=-1)
    accuracy = (predictions == labels).float().mean().item()
    cross_entropy = loss_fn(logits, labels).item()
    return {"accuracy": accuracy, "cross_entropy": cross_entropy}

# por ejemplo:
predicciones_ = [[1.2, -1.2], [-.7, .7], [.9, -.9]] # 3 muestras, dos logits por muestra
labels_ = [0, 1, 0] # 3 muestras, 1 label por muestra
compute_metrics(predicciones_, labels_)

{'accuracy': 1.0, 'cross_entropy': 0.15341037511825562}

In [38]:
ffn_model = ffn_model.to(device)

In [39]:
# El loop de entrenamiento:
from tqdm import tqdm

def validate(model, loader):
    """Función de evaluación en dataset de validación para ser ejecutada
    cada N steps de entrenamiento.
    Args:
        model: modelo
        loader: loader de validación
    Returns:
        Un diccionario con las métricas (nombre -> valor)
    """
    all_labels = []
    all_logits = []
    model.eval()
    with torch.inference_mode():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            inputs_lengths = batch["inputs_lengths"].to(device)
            labels = batch["labels"].to(device)
            logits = model(input_ids, inputs_lengths)
            all_labels.append(labels)
            all_logits.append(logits)
    all_labels = torch.cat(all_labels)
    all_logits = torch.cat(all_logits)
    metrics = compute_metrics(all_logits, all_labels)
    return metrics


def train_epoch(model, optimizer, train_loader, val_loader, best_val_acc, eval_steps=50):
    """Entrenar el modelo un epoch.
    Args:
        model: modelo
        optimizer: optimizador
        train_loader: loader de entrenamiento
        val_loader: loader de validación
        best_val_acc: mejor valor de accuracy en validación
        eval_steps: número de steps entre evaluaciones
    Returns:
        Mejor valor de accuracy en validación
    """
    model.train()
    steps_done = 0
    total_loss = 0
    n_steps = len(train_loader)
    for batch in tqdm(train_loader, total=n_steps, position=0):
        input_ids = batch["input_ids"].to(device)
        inputs_lengths = batch["inputs_lengths"].to(device)
        labels = batch["labels"].to(device)
        optimizer.zero_grad()
        logits = model(input_ids, inputs_lengths)
        batch_loss = loss_fn(logits, labels)
        batch_loss.backward()
        optimizer.step()
        total_loss += batch_loss.item()
        steps_done += 1
        if steps_done % eval_steps == 0:
            metrics = validate(model, val_loader)
            train_loss = total_loss / steps_done
            print(f"\n[steps={steps_done}] train_loss: {train_loss:.4f}, val_acc: {metrics['accuracy']:.4f}")
            if metrics["accuracy"] > best_val_acc:
                best_val_acc = metrics["accuracy"]
                print(f">>> New best accuracy: {best_val_acc:.4f}")
                torch.save(model.state_dict(), "best_model.pt")
            model.train()
    return best_val_acc


def train(model, optimizer, train_loader, val_loader, n_epochs):
    """Entrena el modelo durante n_epochs.
    Args:
        model: modelo
        optimizer: optimizador
        train_loader: loader de entrenamiento
        val_loader: loader de validación
        n_epochs: número de epochs
    """
    best_val_acc = -999
    for epoch in range(n_epochs):
        print(f"Epoch {epoch} / {n_epochs}")
        best_val_acc = train_epoch(
            model, optimizer, train_loader, val_loader, best_val_acc=best_val_acc
        )
        print("-"*70)

**PREGUNTA**: ¿para qué sirve torch.inference_mode()? ¿para qué sirve model.eval()?

In [40]:
n_epochs = 3
optimization_steps = len(train_loader) * n_epochs
print(f"epochs = {n_epochs}, total steps = {optimization_steps}")

epochs = 3, total steps = 801


In [41]:
torch.manual_seed(33)
train(ffn_model, optimizer, train_loader, val_loader, n_epochs)

Epoch 0 / 3


 21%|██        | 55/267 [00:02<00:09, 21.51it/s]


[steps=50] train_loss: 0.6850, val_acc: 0.5863
>>> New best accuracy: 0.5863


 39%|███▉      | 105/267 [00:04<00:06, 25.65it/s]


[steps=100] train_loss: 0.6766, val_acc: 0.6032
>>> New best accuracy: 0.6032


...


[steps=200] train_loss: 0.2553, val_acc: 0.7439


 94%|█████████▍| 252/267 [00:07<00:00, 26.19it/s]


[steps=250] train_loss: 0.2611, val_acc: 0.7448


100%|██████████| 267/267 [00:07<00:00, 35.23it/s]

----------------------------------------------------------------------


## Evaluación en test

Cargamos el mejor modelo y lo evaluamos en el set de test.


In [42]:
best_model = FFNModel(
    vocab_size=len(token2idx),
    embedding_dim=wv_model.vector_size,
    pad_idx=token2idx["<pad>"],
    hidden_size=128,
    num_labels=dataset["train"].features["label"].num_classes,
    embedding_matrix=embedding_matrix
)
best_model.load_state_dict(torch.load("best_model.pt"))
best_model = best_model.to(device)

<ipython-input-42-c8133c2d5f7d>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load("best_model.pt"))


In [43]:
def eval_model(model, loader):
    """Evalúa el modelo en un dataset dado.
    Args:
        model: modelo
        loader: loader del dataset de evaluación
    Returns:
        Un diccionario con las métricas (nombre -> valor)
    """
    model.eval()
    metrics = validate(model, loader)
    return metrics

print("Resultados del mejor modelo en los datos de test:")
test_metrics = eval_model(best_model, test_loader)
for k, v in test_metrics.items():
    print(f"{k:16s}= {v:.4f}")

Resultados del mejor modelo en los datos de test:
accuracy        = 0.7852
cross_entropy   = 0.4476


**PREGUNTA**: ¿cuán buenos son los resultados? ¿cómo podemos saberlo?